In [1]:
# source = https://pymupdf.readthedocs.io/en/latest/recipes-text.html#how-to-extract-all-document-text

import pymupdf
fname = "content.pdf"
with pymupdf.open(fname) as doc:  # open document
    text = chr(12).join([page.get_text() for page in doc])
# write as a binary file to support non-ASCII characters

import unicodedata

def is_pua(c):
    return unicodedata.category(c) == 'Co'


final = "".join([char for char in text if not is_pua(char)])

with open("content.txt", "w") as f:
    f.write(final)
final

"Scholarships:\n\nOver 450 students have received scholarships up to Rs. 3.3 cr\n\nScholarship schemes are divided into 3 categories based on annual family income:\no\nCategory 1: 100% financial assistance for income under Rs. 4 lakh and possessing\nthe National Food Security card\no\nCategory 2: 50% financial assistance for income up to Rs. 2.50 lakh\no\nCategory 3: 25% financial assistance for income between Rs. 2.50-6 lakh\n\nScholarships are also offered based on JEE ranking, 12th board result, and personal interview\n\nStudents get additional monetary benefits from the Delhi state government\nPlacements and Industry Partnerships:\n\nUpcoming companies visiting campus for recruitment include Amazon, Grofers, Network 18,\nCyber group, Appventures, Appsgardz, Mirketa\n\nExperiential learning partnerships with companies like Oracle Academy, Autodesk, Amazon\nWeb Services, CETPA INFOTECH, CISCO Networking Academy, Shape my Skills, NIIT, Halp\nTechnologies\nUniversity Accreditation:\n\n

In [2]:
# source = https://www.restack.io/p/text-chunking-answer-chatbot-cat-ai

from typing import List  

def word_splitter(source_text: str) -> List[str]:  
    import re  
    source_text = re.sub("\s+", " ", source_text)  # Replace multiple whitespaces  
    return re.split("\s", source_text)  # Split by single whitespace 

def get_chunks_fixed_size_with_overlap(text: str, chunk_size: int, overlap_fraction: float) -> List[str]:  
    text_words = word_splitter(text)  
    overlap_int = int(chunk_size * overlap_fraction)  
    chunks = []  
    for i in range(0, len(text_words), chunk_size):  
        chunk_words = text_words[max(i - overlap_int, 0): i + chunk_size]  
        chunk = " ".join(chunk_words)  
        chunks.append(chunk)  
    return chunks 

chunks = get_chunks_fixed_size_with_overlap(final, 9999, overlap_fraction=0.2)   
chunks


<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\chana\AppData\Local\Temp\ipykernel_7576\3655982238.py:7: SyntaxWarning: invalid escape sequence '\s'
  source_text = re.sub("\s+", " ", source_text)  # Replace multiple whitespaces
C:\Users\chana\AppData\Local\Temp\ipykernel_7576\3655982238.py:8: SyntaxWarning: invalid escape sequence '\s'
  return re.split("\s", source_text)  # Split by single whitespace


["Scholarships: Over 450 students have received scholarships up to Rs. 3.3 cr Scholarship schemes are divided into 3 categories based on annual family income: o Category 1: 100% financial assistance for income under Rs. 4 lakh and possessing the National Food Security card o Category 2: 50% financial assistance for income up to Rs. 2.50 lakh o Category 3: 25% financial assistance for income between Rs. 2.50-6 lakh Scholarships are also offered based on JEE ranking, 12th board result, and personal interview Students get additional monetary benefits from the Delhi state government Placements and Industry Partnerships: Upcoming companies visiting campus for recruitment include Amazon, Grofers, Network 18, Cyber group, Appventures, Appsgardz, Mirketa Experiential learning partnerships with companies like Oracle Academy, Autodesk, Amazon Web Services, CETPA INFOTECH, CISCO Networking Academy, Shape my Skills, NIIT, Halp Technologies University Accreditation: IP University has accreditation 

In [3]:
#!pip install -U -q "google-generativeai>=0.7.2"
import textwrap # for formatting the prompt
import numpy as np # for implementing RAG
import pandas as pd # to handle the data and embeddings

import google.generativeai as genai # to call the api

from IPython.display import Markdown # to access the response from the chatbot in proper format

C:\Users\chana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
genai.configure(api_key="Your api key")

In [5]:
Doc = []
for i in chunks:
    doc_temp = {}
    doc_temp["Title"] = i.split()[0]
    doc_temp["content"] = i
    Doc.append(doc_temp)
Doc

[{'Title': 'Scholarships:',
  'content': "Scholarships: Over 450 students have received scholarships up to Rs. 3.3 cr Scholarship schemes are divided into 3 categories based on annual family income: o Category 1: 100% financial assistance for income under Rs. 4 lakh and possessing the National Food Security card o Category 2: 50% financial assistance for income up to Rs. 2.50 lakh o Category 3: 25% financial assistance for income between Rs. 2.50-6 lakh Scholarships are also offered based on JEE ranking, 12th board result, and personal interview Students get additional monetary benefits from the Delhi state government Placements and Industry Partnerships: Upcoming companies visiting campus for recruitment include Amazon, Grofers, Network 18, Cyber group, Appventures, Appsgardz, Mirketa Experiential learning partnerships with companies like Oracle Academy, Autodesk, Amazon Web Services, CETPA INFOTECH, CISCO Networking Academy, Shape my Skills, NIIT, Halp Technologies University Accredi

In [6]:
df = pd.DataFrame(Doc)
df.columns = ['Title', 'Text']
df

,Title,Text
0,Scholarships:,Scholarships: Over 450 students have received ...


In [7]:
model = 'models/embedding-001'
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,Embeddings
0,Scholarships:,Scholarships: Over 450 students have received ...,"[0.06562772, -0.032361746, -0.054728627, 0.021..."


In [8]:
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model= 'models/embedding-001',
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text']

In [9]:
query = "What can you tell me about the college Delhi Technical campus ?"
passage = find_best_passage(query, df)
passage

"Scholarships: Over 450 students have received scholarships up to Rs. 3.3 cr Scholarship schemes are divided into 3 categories based on annual family income: o Category 1: 100% financial assistance for income under Rs. 4 lakh and possessing the National Food Security card o Category 2: 50% financial assistance for income up to Rs. 2.50 lakh o Category 3: 25% financial assistance for income between Rs. 2.50-6 lakh Scholarships are also offered based on JEE ranking, 12th board result, and personal interview Students get additional monetary benefits from the Delhi state government Placements and Industry Partnerships: Upcoming companies visiting campus for recruitment include Amazon, Grofers, Network 18, Cyber group, Appventures, Appsgardz, Mirketa Experiential learning partnerships with companies like Oracle Academy, Autodesk, Amazon Web Services, CETPA INFOTECH, CISCO Networking Academy, Shape my Skills, NIIT, Halp Technologies University Accreditation: IP University has accreditation f

In [10]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a chatbot for the college Delhi technical campus, who is on the website of the college and you will be answering the questions of the incoming
  students. Use the passage to answer the questions. Also don't repeat information and try to make your response more human like.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [11]:
prompt = make_prompt(query, passage)
print(prompt)

You are a chatbot for the college Delhi technical campus, who is on the website of the college and you will be answering the questions of the incoming
  students. Use the passage to answer the questions. Also don't repeat information and try to make your response more human like.
  QUESTION: 'What can you tell me about the college Delhi Technical campus ?'
  PASSAGE: 'Scholarships: Over 450 students have received scholarships up to Rs. 3.3 cr Scholarship schemes are divided into 3 categories based on annual family income: o Category 1: 100% financial assistance for income under Rs. 4 lakh and possessing the National Food Security card o Category 2: 50% financial assistance for income up to Rs. 2.50 lakh o Category 3: 25% financial assistance for income between Rs. 2.50-6 lakh Scholarships are also offered based on JEE ranking, 12th board result, and personal interview Students get additional monetary benefits from the Delhi state government Placements and Industry Partnerships: Upcomin

In [12]:
model = genai.GenerativeModel('models/gemini-1.5-flash')
gemini_chat = model.start_chat()
answer = gemini_chat.send_message(prompt)
answer

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Delhi Technical Campus sounds like a great place to study! It's part of IP University, which has some pretty impressive accreditations like a QS World Ranking, NAAC A++ rating, and NIRF 74 ranking.  They have a strong focus on practical learning with partnerships with companies like Oracle Academy, Autodesk, and Amazon Web Services.  \n\nYou'll have lots of opportunities to get hands-on experience through workshops, technical activities, and hackathons. There are also technical and cultural societies to join so you can explore your interests outside the classroom.  \n\nThe placement record is pretty good too, with a high placement rate and average salary of 6-7 LPA. You can get help from their placement cell to land a job at companies like IBM, HCL, TCS, Cog

In [13]:
print("Question : ",query)
Markdown(answer.text)

Question :  What can you tell me about the college Delhi Technical campus ?


Delhi Technical Campus sounds like a great place to study! It's part of IP University, which has some pretty impressive accreditations like a QS World Ranking, NAAC A++ rating, and NIRF 74 ranking.  They have a strong focus on practical learning with partnerships with companies like Oracle Academy, Autodesk, and Amazon Web Services.  

You'll have lots of opportunities to get hands-on experience through workshops, technical activities, and hackathons. There are also technical and cultural societies to join so you can explore your interests outside the classroom.  

The placement record is pretty good too, with a high placement rate and average salary of 6-7 LPA. You can get help from their placement cell to land a job at companies like IBM, HCL, TCS, Cognizant, Deloitte Capgemini, Infosys, WIPRO, Amazon, and Adobe.

If you're interested in entrepreneurship, there's an Innovation Council / E-Cell that can help you turn your ideas into reality. They offer mentorship programs, workshops, and even funding opportunities for startups.  

Overall, Delhi Technical Campus seems like a good choice for students looking for a well-rounded education with good career prospects.


In [14]:
query = "Which one is releated to AI ?"
passage = find_best_passage(query=query, dataframe=df)
prompt = make_prompt(query,passage)
answer = gemini_chat.send_message(prompt)
Markdown(answer.text)

Well, it looks like there are a couple of things related to AI at Delhi Technical Campus! 

First, you can actually major in **Artificial Intelligence and Machine Learning (AI&ML)** or **AI & Data Science**! These are both B.Tech programs, so you'd be getting a great foundation in the field.  

Second, they also offer value-added courses in **AI and Machine Learning**, so even if you choose another program, you can still learn more about it.  Plus, there's an **MOU with i-STEP** to provide international masterclass/certificate courses on AI, Data Science, Cyber security, and Entrepreneurship from Deakin University and UNSW University in Australia. 

So, yeah, Delhi Technical Campus seems to take AI seriously!  If you're interested in the field, it's definitely a place to consider. 


In [15]:
query = "Which better among them ?"
passage = find_best_passage(query=query, dataframe=df)
prompt = make_prompt(query,passage)
answer = gemini_chat.send_message(prompt)
Markdown(answer.text)

Hmm, you didn't tell me what you're comparing!  Can you tell me which two things you want me to compare?   

For example, are you asking about:

* **Two different B.Tech programs?** (like Computer Science Engineering vs. Artificial Intelligence and Machine Learning)
* **Hostel options?** (like triple sharing vs. double sharing)
* **Scholarship categories?** (like Category 1 vs. Category 2)

Once you tell me what you're interested in comparing, I can give you some insights based on the information provided. 
